<div class="header">
  <img src="img/kg_logo_white_side.png" alt="logo" style="width: 300px;"/>
  <h1>Transaction Monitoring with Graph Neural Networks</h1>
</div>

The Elliptic Data Set maps Bitcoin transactions to real entities belonging to licit categories (exchanges, wallet providers, miners, licit services, etc.) versus illicit ones (scams, malware, terrorist organizations, ransomware, Ponzi schemes, etc.). The task on the dataset is to classify the illicit and licit nodes in the graph.
We will use Graph Neural Networks (GraphSAGE) to perform the node classification task. This demo shows the end-to-end pipeline that can be done directly on the Katana Graph platform. 

-------

## Setup

In [2]:
import os                                                                        
import time                                                                      
import json
import uuid
import pandas as pd
import numpy
import argparse
from timeit import default_timer as timer
import warnings
warnings.filterwarnings('ignore')
from katana import remote
from katana.remote import import_data
from katana.remote.import_data import Operation
import dask.dataframe as dd
os.environ["KATANA_SERVER_ADDRESS"] = "localhost:8080"

## Dask Dataframe Import

Use Python dataframes interface to prepare and generate graph

In [3]:
tx_classes = "gs://katana-demo-datasets/fsi/solution_raw_data/elliptic/elliptic_txs_classes.csv"
tx_edges = "gs://katana-demo-datasets/fsi/solution_raw_data/elliptic/elliptic_txs_edgelist.csv"
tx_features = "gs://katana-demo-datasets/fsi/solution_raw_data/elliptic/elliptic_txs_features.csv"

In [4]:
feat_col_names = ["txId", "timestamp"]
local_feats_name = [f"local_feat_{i}" for i in range(2,95)]
agg_feats_name = [f"agg_feat_{i}" for i in range(95,167)]
feat_col_names.extend(local_feats_name)
feat_col_names.extend(agg_feats_name)

In [5]:
feat_types = {
    "class": "string",
    "timestamp": "string", 
    "target": "float",
    "node_type": "string"
}
local_cols = {}
for i in range(2,95):
    local_cols[f"local_feat_{i}"] = "float"
agg_cols = {}
for i in range(95,167):
    agg_cols[f"agg_feat_{i}"] = "float"
feat_types.update(local_cols)
feat_types.update(agg_cols)

In [6]:
%%time
classes = dd.read_csv(tx_classes)
edges = dd.read_csv(tx_edges)
features = dd.read_csv(tx_features, header=None, names=feat_col_names)

CPU times: user 311 ms, sys: 71.3 ms, total: 382 ms
Wall time: 3.17 s


In [7]:
classes['target'] = classes['class'].map({'unknown': -1.0, '1': 1.0, '2': 0.0})
classes['node_type'] = classes['class'].map({'unknown': 'Unclassified_Txn', '1': 'Classified_Txn', '2': 'Classified_Txn'})

Some of the elliptic transactions are classified as either licit (0) or illicit (1). We can filter for only classified transactions for training

In [8]:
classes.head()

,txId,class,target,node_type
0,230425980,unknown,-1.0,Unclassified_Txn
1,5530458,unknown,-1.0,Unclassified_Txn
2,232022460,unknown,-1.0,Unclassified_Txn
3,232438397,2,0.0,Classified_Txn
4,230460314,unknown,-1.0,Unclassified_Txn


In [9]:
edges.head()

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [10]:
nodes = features.merge(classes)

In [11]:
nodes.head()

,txId,timestamp,local_feat_2,local_feat_3,local_feat_4,local_feat_5,local_feat_6,local_feat_7,local_feat_8,local_feat_9,...,agg_feat_160,agg_feat_161,agg_feat_162,agg_feat_163,agg_feat_164,agg_feat_165,agg_feat_166,class,target,node_type
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,unknown,-1.0,Unclassified_Txn
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,unknown,-1.0,Unclassified_Txn
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,unknown,-1.0,Unclassified_Txn
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,2,0.0,Classified_Txn
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,unknown,-1.0,Unclassified_Txn


There is a 4 node cluster running on GCP, which is why we select 4 partitions. As the data scales, we can increase the number of machines in the cluster and the number of partitions in our graph

In [12]:
graph = remote.Client(disable_version_check=False).create_graph(
    num_partitions=4
)
print("graph id:", graph.graph_id)

graph id: cVxD69rsahp5Y4M4U1HoARLVatbxZG5JviszizMRFsz


In [13]:
%%time
reverse_edges=True
with import_data.DataFrameImporter(graph) as df_importer:   
    
    df_importer.nodes_dataframe(nodes,
                            id_column="txId",
                            id_space="transaction", 
                            property_columns=feat_types,
                            label_column="node_type")
    
    df_importer.edges_dataframe(edges,
                            source_id_space="transaction",
                            destination_id_space="transaction",
                            source_column="txId1",
                            destination_column="txId2",
                            type="tx_flow")
    if reverse_edges:
        df_importer.edges_dataframe(edges,
                        source_id_space="transaction",
                        destination_id_space="transaction",
                        source_column="txId2",
                        destination_column="txId1",
                        type="rev_tx_flow")
    

          0/? [?op/s]

In [13]:
print(f"Number of nodes: {graph.num_nodes():,}")
print(f"Number of edges: {graph.num_edges():,}")

          0/? [?op/s]

2022-07-25 14:49:03,118 - distributed.diskutils - INFO - Found stale lock file and directory '/home/gsteck_katanagraph_com/solutions/fsi/notebooks/aml/dask-worker-space/worker-lj6zho8s', purging
2022-07-25 14:49:03,127 - distributed.diskutils - INFO - Found stale lock file and directory '/home/gsteck_katanagraph_com/solutions/fsi/notebooks/aml/dask-worker-space/worker-2fu75wr9', purging
2022-07-25 14:49:03,140 - distributed.diskutils - INFO - Found stale lock file and directory '/home/gsteck_katanagraph_com/solutions/fsi/notebooks/aml/dask-worker-space/worker-96kkd5sg', purging
2022-07-25 14:49:03,152 - distributed.diskutils - INFO - Found stale lock file and directory '/home/gsteck_katanagraph_com/solutions/fsi/notebooks/aml/dask-worker-space/worker-_4k2rx7j', purging
2022-07-25 14:49:03,163 - distributed.diskutils - INFO - Found stale lock file and directory '/home/gsteck_katanagraph_com/solutions/fsi/notebooks/aml/dask-worker-space/worker-bvttwj5b', purging
2022-07-25 14:49:03,174 -

Number of nodes: 203,769


          0/? [?op/s]

Number of edges: 468,710


## Visualization

See all relationships from transaction

In [16]:
%%time
graph.query("""
MATCH (tx1)-[:tx_flow]->(tx2)
RETURN tx1, tx2
LIMIT 500
""", contextualize=True).visualize()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 6.99 s, sys: 996 ms, total: 7.98 s
Wall time: 1min 1s


GraphVizWidget(edges=[{'from': 0, 'to': 1, 'label': 'tx_flow', 'type': 'tx_flow', 'properties': {}}, {'from': …

## Feature Initialization

To initialize features for the GNN, we will combine all node properties into a feature vector and save as a new feature on the graph. Katana Graph supports saving binary feature vectors as individual properties on the graph. In this case, we save 3 different feature vectors: 

1. ```local_feats``` - raw features provided for each transaction
2. ```agg_feats```  - aggregated features from each node's neighborhood
3. ```h_init``` - both ```local_feats``` + ```agg_feats``` combined into one feature. This will be the starting point for our GNN

In [14]:
%%time
def run_feature_init(g): 
    import sys, os
    import numpy as np
    sys.path.append(os.path.join("/home/gsteck_katanagraph_com/solutions/fsi/src"))
    from katana_ai import get_node_property_list, visualize_embeddings, train_test_split_mask, save_features_to_graph
    # extract features
    local_feats = get_node_property_list(g, property_list=local_feats_name)
    agg_feats = get_node_property_list(g, property_list=agg_feats_name)
    feat_vec = np.concatenate([local_feats, agg_feats], axis=-1)
    # save new features vector to graph
    g = save_features_to_graph(g, feat_vec, feature_name="h_init")
    g = save_features_to_graph(g, local_feats, feature_name="local_feats")
    g = save_features_to_graph(g, agg_feats, feature_name="agg_feats")
    # create train/test split mask
    g = train_test_split_mask(g, train_test_validation_split=[0.8, 0.15, 0.05])
    g.write()

graph.run(lambda g: run_feature_init(g))

          0/? [?op/s]

CPU times: user 4.72 s, sys: 983 ms, total: 5.7 s
Wall time: 27.2 s


## Analyze Features

Generate run ID for Tensorboard

In [15]:
run_id = uuid.uuid4().hex

Analyze initial feature vectors in Tensorboard on ```Projector``` tab. Features can be visualized using T-SNE / UMAP plots. 

In [17]:
def analyze_features(g): 
    import sys, os
    sys.path.append(os.path.join("/home/gsteck_katanagraph_com/solutions/fsi/src"))
    from katana_ai import visualize_embeddings
    from torch.utils.tensorboard import SummaryWriter
    writer = SummaryWriter(f"/tmp/tensorboard/elliptic-embed-init-{run_id}")
    # analyze features in tensorboard
    visualize_embeddings(g, writer, feature_name="h_init", target_name="target", filter_node_type="Classified_Txn", sample_size=2000)
    visualize_embeddings(g, writer, feature_name="local_feats", target_name="target", filter_node_type="Classified_Txn", sample_size=2000)
    visualize_embeddings(g, writer, feature_name="agg_feats", target_name="target", filter_node_type="Classified_Txn", sample_size=2000)
    
graph.run(lambda g: analyze_features(g))
print(f"See results at https://demo-finance-tensorboard.katanagraph.com/")
print(f"Run ID: elliptic-embed-init-{run_id}")

          0/? [?op/s]

OperationError: unexpected failure: worker output:

[1,0]<stdout>:{"host":0,"offset_ms":2890496,"log":{"msg":"katana operation root","timestamp_us":1658758049211769,"max_mem_gb":1.865,"mem_gb":1.672,"arrow_mem_gb":0.029,"sent_gb":0.015,"recv_gb":0.007},"host_data":{"hosts":4,"hostname":"katana-compute-1","hardware_threads":32,"pid":3151,"ram_gb":251},"span_data":{"span_name":"katana operation root","span_id":"3a74e1d2871c2798","parent_id":"null"},"trace_id":"d0917b6bb60dbac9cba70686b6472e35"}
[1,0]<stdout>:{"host":0,"offset_ms":2890500,"log":{"msg":"katana host root","timestamp_us":1658758049215567,"max_mem_gb":1.865,"mem_gb":1.672,"arrow_mem_gb":0.029,"sent_gb":0.015,"recv_gb":0.007},"span_data":{"span_name":"katana host root","span_id":"9b4bbec3de60bb0e","parent_id":"3a74e1d2871c2798"},"trace_id":"d0917b6bb60dbac9cba70686b6472e35"}
[1,0]<stdout>:{"host":0,"offset_ms":2890500,"tags":[{"name":"operation_id","value":"2E6PgNkTTxgQbsnKmRVwu6sUyDPQ9HCfFHLscc5vuhUb-51GQwyez4NDj7a7wS"}],"spa

## GNN Training

In [16]:
args = argparse.Namespace(
    feat_name="h_init",
    label_name="target",
    label_dtype=numpy.float32,
    split_name="train_test_val_mask",
    distributed_execution=True,
    tensorboard_dir=f"/tmp/tensorboard/elliptic-remote-{run_id}",
    model_dir="/tmp/models",
    katana_ai_dir="/home/gsteck_katanagraph_com/solutions/fsi/src",
    pred_node_label="Classified_Txn",
    pred_node_label_prop="node_type",
    pos_weight=8,
    in_dim=165,
    hidden_dim=256,
    train_fan_in="100,100,100,100",
    test_fan_in="100,100,100,100",
    num_layers=4,
    out_dim=1,
    minibatch_size=1024,
    max_minibatches=20,
    lr=0.001,
    dropout=0.2,
    num_epochs=100
)

In [17]:
def run_gnn(graph, args):
    from calendar import c
    import torch
    import numpy
    import katana
    from katana_enterprise.distributed.pytorch import init_workers
    from katana_enterprise.ai.data import PyGNodeSubgraphSampler, SampledSubgraphConfig 
    from katana_enterprise.ai.data import NodeDataLoader
    from torch_geometric.nn import SAGEConv
    from torch.nn.parallel import DistributedDataParallel as torch_DDP
    from torch.utils.tensorboard import SummaryWriter
    import sys, os
    sys.path.append(os.path.join(args.katana_ai_dir))
    from katana_ai import get_split, train_model
    os.environ['MODIN_ENGINE']='python'
    #katana.distributed.initialize()

    katana.set_active_threads(32)
    exec_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # model definition
    class DistSAGE(torch.nn.Module):
        def __init__(self, in_dim, hidden_dim, out_dim, num_layers,
                     dropout):
            super(DistSAGE, self).__init__()

            self.convs = torch.nn.ModuleList()
            self.convs.append(SAGEConv(in_dim, hidden_dim))
            self.bns = torch.nn.ModuleList()
            self.bns.append(torch.nn.BatchNorm1d(hidden_dim))
            for _ in range(num_layers - 2):
                self.convs.append(SAGEConv(hidden_dim, hidden_dim))
                self.bns.append(torch.nn.BatchNorm1d(hidden_dim))
            self.convs.append(SAGEConv(hidden_dim, out_dim))
            self.activation = torch.nn.functional.relu
            self.dropout = torch.nn.Dropout(dropout)

        def reset_parameters(self):
            for conv in self.convs:
                conv.reset_parameters()
            for bn in self.bns:
                bn.reset_parameters()

        def forward(self, data):
            # unpack data loader
            x, edges = data.x, data.adjs
            #x.to(exec_device)
            #edges.to(exec_device)

            for i, conv in enumerate(self.convs):
                # for multilayer, set x_target for each layer
                x_target = x[:data.dest_count[i]]
                x = conv((x, x_target), edges[i])
                if i != len(self.convs) - 1:
                    x = self.bns[i](x)
                    x = self.activation(x)
                    x = self.dropout(x) 
                    embed = x
            return x, embed
    
    # initialize torch mpi process
    main_start = time.time()
    if args.distributed_execution:
        init_workers()

    # tensorboard writer
    writer = SummaryWriter(args.tensorboard_dir)

    # split train / test node idx
    train_nodes = get_split(graph, 0, split_name=args.split_name, node_label=args.pred_node_label, node_label_prop=args.pred_node_label_prop)
    test_nodes = get_split(graph, 1, split_name=args.split_name, node_label=args.pred_node_label, node_label_prop=args.pred_node_label_prop)

    # initialize the multiminibatch sampler
    train_sampler = PyGNodeSubgraphSampler(
        graph, 
        SampledSubgraphConfig(
        layer_fan=[int(fan_in) for fan_in in args.train_fan_in.split(',')], 
            max_minibatches=args.max_minibatches, 
            batch_props_to_pull=args.max_minibatches,
            feat_prop_name=args.feat_name,
            label_prop_name=args.label_name,
            label_dtype=args.label_dtype,
            multilayer_export=True
        )
    )
    
    # test sampler used for evaluation; it samples 100s per hop to simulate getting
    test_sampler = PyGNodeSubgraphSampler(
        graph, 
        SampledSubgraphConfig(
        layer_fan=[int(fan_in) for fan_in in args.train_fan_in.split(',')], 
            max_minibatches=args.max_minibatches, 
            batch_props_to_pull=args.max_minibatches,
            feat_prop_name=args.feat_name,
            label_prop_name=args.label_name,
            label_dtype=args.label_dtype,
            pull_edge_types=args.load_edge_types,
            multilayer_export=True
        )
    )
    
    # shuffle seeds between epochs + balance seed nodes across hosts
    train_dataloader = NodeDataLoader(
        train_sampler, 
        local_batch_size=args.minibatch_size, 
        node_ids=train_nodes,  
        shuffle=True, 
        drop_last=True,
        balance_seeds=True)
    test_dataloader = NodeDataLoader(
        test_sampler, 
        local_batch_size=args.minibatch_size, 
        node_ids=test_nodes, 
        balance_seeds=True)

    # model initialization
    model = DistSAGE(
        in_dim=args.in_dim, 
        hidden_dim=args.hidden_dim, 
        out_dim=args.out_dim, 
        num_layers=args.num_layers,
        dropout=args.dropout
    ).to(exec_device)

    if args.distributed_execution:
        model = torch_DDP(model)
    
    # optimizer and loss fn
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    loss_function = torch.nn.BCEWithLogitsLoss(pos_weight=torch.Tensor([args.pos_weight]))
    
    # train model
    train_model(model, loss_function, optimizer, writer, train_dataloader, test_dataloader, args)
    
    # save model
    #ts = time.time()
    #torch.save(model.state_dict(), os.path.join(args.model_dir, 'graph_sage.'+str(ts)+'.pth'))


Track results at https://demo-finance-tensorboard.katanagraph.com/
Run ID: elliptic-remote-a3e35d51259649169df68417474099fe


          0/? [?op/s]

OperationError: unexpected failure: worker output:

[1,0]<stdout>:{"host":0,"offset_ms":223683,"log":{"msg":"katana operation root","timestamp_us":1658760575235566,"max_mem_gb":1.774,"mem_gb":1.590,"arrow_mem_gb":0.000,"sent_gb":0.015,"recv_gb":0.007},"host_data":{"hosts":4,"hostname":"katana-compute-3","hardware_threads":32,"pid":11309,"ram_gb":251},"span_data":{"span_name":"katana operation root","span_id":"605d3e05b438341d","parent_id":"null"},"trace_id":"f8245d5f20782cdcad1ac438a55efbe8"}
[1,0]<stdout>:{"host":0,"offset_ms":223683,"log":{"msg":"katana host root","timestamp_us":1658760575235922,"max_mem_gb":1.774,"mem_gb":1.590,"arrow_mem_gb":0.000,"sent_gb":0.015,"recv_gb":0.007},"span_data":{"span_name":"katana host root","span_id":"0caf830c7fcc9e1c","parent_id":"605d3e05b438341d"},"trace_id":"f8245d5f20782cdcad1ac438a55efbe8"}
[1,0]<stdout>:{"host":0,"offset_ms":223683,"tags":[{"name":"operation_id","value":"5pk3qApNdztVXGpZSdH3Fgzp6c85pEBdbHSNTqDgT772-xdFDBq4DAkz82ZC5"}],"span_d